<a href="https://colab.research.google.com/github/wjleece/anthropic-prompts/blob/main/Anthropic_Customer_Service_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install anthropic

In [69]:
import anthropic
import re
import json
from datetime import datetime
from typing import Dict, List, Any, Optional, Union
from fuzzywuzzy import process, fuzz
from google.colab import userdata

API_KEY = userdata.get('ANTHROPIC_API_KEY')
MODEL_NAME = "claude-3-7-sonnet-latest"
client = anthropic.Anthropic(api_key=API_KEY)

In [65]:
system_prompt = """
You are a helpful customer service assistant for an e-commerce system.

When responding to the user, use the conversation context to maintain continuity.
- If a user refers to "my order" or similar, use the context to determine which order they're talking about.
- If they mention "that product" or use other references, check the context to determine what they're referring to.
- Always prioritize recent context over older context when resolving references.

The conversation context will be provided to you with each message. This includes:
- Previous questions and answers
- Recently viewed customers, products, and orders
- Recent actions taken (like creating orders, updating products, etc.)

Keep responses friendly, concise, and helpful. If you're not sure what a user is referring to, ask for clarification.
"""

In [14]:
tools = []

In [15]:
def get_completion(prompt: str):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        tools=tools,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [ ]:
prompt="Yo Claude, wuzzup?"

print(get_completion(prompt))

Hey there! Not much on my end, just hanging out in the digital realm, ready to help with whatever you need. What's going on with you today? Anything I can help you with or chat about?


In [38]:
# Global customer storage
customers = {
    "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
    "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
}

#Global product storage
products = {
    "P1": {"name": "Widget A", "description": "A simple widget. Very compact.", "price": 19.99, "inventory_count": 999},
    "P2": {"name": "Gadget B", "description": "A powerful gadget. It spins.", "price": 49.99, "inventory_count":200}
}

# Global order storage
orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }

In [66]:
tools = [
    {
        "name": "create_customer",
        "description": "Adds a new customer to the database. Includes customer ID, name, email, and (optional) phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the customer."
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the customer."
                },
                "phone": {
                    "type": "string",
                    "description": "The phone number of the customer."
                }
            },
            "required": ["name", "email"]
        }
    },
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and (optional) phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "create_product",
        "description": "Adds a new product to the product database. Includes ID, name, description, price, and initial inventory count.",
        "input_schema": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the product."
                },
                "description": {
                    "type": "string",
                    "description": "A description of the product."
                },
                "price": {
                    "type": "number",
                    "description": "The price of the product."
                },
                "inventory_count": {
                    "type": "integer",
                    "description": "The amount of the product that is currently in inventory."
                }
            },
            "required": ["name", "description", "price", "inventory_count"]
        }
    },
    {
        "name": "update_product",
        "description": "Updates an existing product with new information. Only fields that are provided will be updated; other fields remain unchanged.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id": {
                    "type": "string",
                    "description": "The unique identifier for the product to update."
                },
                "name": {
                    "type": "string",
                    "description": "The new name for the product (optional)."
                },
                "description": {
                    "type": "string",
                    "description": "The new description for the product (optional)."
                },
                "price": {
                    "type": "number",
                    "description": "The new price for the product (optional)."
                },
                "inventory_count": {
                    "type": "integer",
                    "description": "The new inventory count for the product (optional)."
                }
            },
            "required": ["product_id"]
        }
    },
    {
        "name": "get_product_info",
        "description": "Retrieves product information based on product ID or product name (with fuzzy matching for misspellings). Returns product details including name, description, price, and inventory count.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id_or_name": {
                    "type": "string",
                    "description": "The product ID or name (can be approximate)."
                }
            },
            "required": ["product_id_or_name"]
        }
    },
    {
        "name": "list_all_products",
        "description": "Lists all available products in the inventory.",
        "input_schema": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "create_order",
        "description": """Creates an order using the product's current price.
                      If requested quantity exceeds available inventory, no order is created and available quantity is returned.
                      Orders can only be created for products that are in stock.
                      Supports specifying products by either ID or name with fuzzy matching for misspellings.""",
        "input_schema": {
            "type": "object",
            "properties": {
                "product_id_or_name": {
                    "type": "string",
                    "description": "The ID or name of the product to order (supports fuzzy matching)."
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the product in the order."
                },
                "status": {
                    "type": "string",
                    "description": "The initial status of the order (e.g., 'Processing', 'Shipped')."
                }
            },
            "required": ["product_id_or_name", "quantity", "status"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "update_order_status",
        "description": """Updates the status of an order and adjusts inventory accordingly.
                      Changing to "Shipped" decreases inventory.
                      Changing to "Returned" or "Canceled" from "Shipped" increases inventory.
                      Status can be "Processing", "Shipped", "Delivered", "Returned", or "Canceled".""",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                },
                "new_status": {
                    "type": "string",
                    "description": "The new status to set for the order.",
                    "enum": ["Processing", "Shipped", "Delivered", "Returned", "Canceled"]
                }
            },
            "required": ["order_id", "new_status"]
        }
    },
    {
        "name": "cancel_order",
        "description": """Cancels an order based on the provided order ID. Sets the order status to "Canceled" and returns a confirmation message if the cancellation is successful. If the order was already shipped, this will also return the items to inventory.""",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [67]:
# Customer functions
def create_customer(name, email, phone):
    new_id = f"C{len(customers) + 1}"  # Simple ID generator like C3, C4, ...
    customers[new_id] = {"name": name, "email": email, "phone": phone}
    return new_id, customers[new_id]

def get_customer_info(customer_id):
    return customers.get(customer_id, "Customer not found")

# Product functions
def create_product(name, description, price, inventory_count):
    new_id = f"P{len(products) + 1}"  # Simple ID generator
    products[new_id] = {
        "name": name,
        "description": description,
        "price": float(price),
        "inventory_count": int(inventory_count)
    }
    return new_id, products[new_id]

def update_product(product_id, **update_params):
    """
    Updates a product with the provided parameters. Only updates fields that are provided,
    leaving other fields unchanged.
    """
    # Check if product exists
    if product_id not in products:
        return {
            "status": "error",
            "message": f"Product {product_id} not found"
        }

    # Get the current product data
    product = products[product_id]

    # Track which fields were updated
    updated_fields = []

    # Update only the fields that were provided
    if "name" in update_params and update_params["name"] is not None:
        product["name"] = update_params["name"]
        updated_fields.append("name")

    if "description" in update_params and update_params["description"] is not None:
        product["description"] = update_params["description"]
        updated_fields.append("description")

    if "price" in update_params and update_params["price"] is not None:
        product["price"] = float(update_params["price"])  # Ensure it's stored as a float
        updated_fields.append("price")

    if "inventory_count" in update_params and update_params["inventory_count"] is not None:
        product["inventory_count"] = int(update_params["inventory_count"])  # Ensure it's stored as an integer
        updated_fields.append("inventory_count")

    # If no fields were updated
    if not updated_fields:
        return {
            "status": "warning",
            "message": "No fields were updated. Please provide at least one field to update.",
            "product": product
        }

    # Return success response
    return {
        "status": "success",
        "message": f"Product {product_id} updated successfully. Updated fields: {', '.join(updated_fields)}",
        "product_id": product_id,
        "updated_fields": updated_fields,
        "product": product
    }

# Modified function to get product ID from either direct ID or fuzzy name matching
def get_product_id(product_identifier):
    """
    Get product ID either directly (if it's already an ID) or by fuzzy matching the name.

    Args:
        product_identifier (str): Either a product ID (e.g., "P1") or a product name (e.g., "Widget A")

    Returns:
        str: Product ID if found, None otherwise
    """
    # If it's already a product ID format
    if product_identifier in products:
        return product_identifier

    # Try fuzzy matching by name
    product_id, _ = find_product_by_name(product_identifier)
    return product_id

def find_product_by_name(product_name, min_similarity=70):
    """
    Find a product by name using fuzzy string matching.
    Returns the product ID and the matched product, or None if no match is found.

    Args:
        product_name (str): The name to search for
        min_similarity (int): Minimum similarity score (0-100) to consider a match

    Returns:
        tuple: (product_id, product_dict) or (None, None) if no match
    """
    if not product_name:
        return None, None

    # Get all product names from the products dictionary
    product_choices = [(pid, p["name"]) for pid, p in products.items()]

    if not product_choices:
        return None, None

    # Use fuzzywuzzy to find the best match
    best_match = process.extractOne(
        product_name,
        [name for _, name in product_choices],
        scorer=fuzz.token_sort_ratio
    )

    if best_match and best_match[1] >= min_similarity:
        matched_name = best_match[0]
        # Find the corresponding product ID
        product_id = next((pid for pid, name in product_choices if name == matched_name), None)
        return product_id, products[product_id]

    return None, None

# Order functions
def create_order(product_id_or_name, quantity, status):
    """
    Creates an order using the product's stored price.
    Now supports specifying products by either ID or name.
    """
    # Get product ID (whether directly provided or through fuzzy name matching)
    product_id = get_product_id(product_id_or_name)

    # If we couldn't find a product
    if not product_id:
        return {
            "status": "error",
            "message": f"Product '{product_id_or_name}' not found. Please check the product name or ID and try again."
        }

    # Get product info and price
    product = products[product_id]
    price = product["price"]

    # Check if inventory exists and is sufficient
    if "inventory_count" not in product or product["inventory_count"] == 0:
        return "Out of stock"

    available_quantity = product["inventory_count"]

    # If requested quantity exceeds available inventory
    if quantity > available_quantity:
        return {
            "status": "partial_availability",
            "message": f"Insufficient inventory. Only {available_quantity} units of {product['name']} are available.",
            "available_quantity": available_quantity,
            "requested_quantity": quantity,
            "product_name": product['name']
        }

    # If we have enough inventory and status is "Shipped", update inventory
    if status == "Shipped":
        # Use update_product to decrease inventory
        new_inventory = product["inventory_count"] - quantity
        update_product(product_id, inventory_count=new_inventory)

    # Create the new order
    new_id = f"O{len(orders) + 1}"
    orders[new_id] = {
        "id": new_id,
        "product": product_id,
        "product_name": product["name"],  # Store product name for easier reference
        "quantity": quantity,
        "price": price,
        "status": status
    }

    return {
        "status": "success",
        "order_id": new_id,
        "order_details": orders[new_id],
        "remaining_inventory": products[product_id]["inventory_count"]
    }

def get_order_details(order_id):
    return orders.get(order_id, "Order not found")

def update_order_status(order_id, new_status):
    """
    Updates the status of an order and adjusts inventory accordingly.
    """
    # Check if the order exists
    if order_id not in orders:
        return "Order not found"

    # Get the current order
    order = orders[order_id]
    old_status = order["status"]
    product_id = order["product"]
    quantity = order["quantity"]

    # If status hasn't changed, do nothing
    if old_status == new_status:
        return {
            "status": "unchanged",
            "message": f"Order {order_id} status unchanged, still {old_status}"
        }

    # Update inventory based on status change
    if product_id in products:
        current_inventory = products[product_id]["inventory_count"]

        # If changing from not-shipped to shipped, decrease inventory
        if new_status == "Shipped" and old_status in ["Processing", "Pending"]:
            if current_inventory < quantity:
                return f"Insufficient inventory. Available: {current_inventory}, Required: {quantity}"

            # Use update_product to decrease inventory
            update_product(product_id, inventory_count=current_inventory - quantity)

        # If changing from shipped to returned, increase inventory
        elif new_status == "Returned" and old_status in ["Shipped", "Delivered"]:
            # Use update_product to increase inventory
            update_product(product_id, inventory_count=current_inventory + quantity)

        # If changing from shipped to canceled, increase inventory
        elif new_status == "Canceled" and old_status == "Shipped":
            # Use update_product to increase inventory
            update_product(product_id, inventory_count=current_inventory + quantity)

    # Update the order status
    orders[order_id]["status"] = new_status

    return {
        "status": "success",
        "message": f"Order {order_id} status updated from {old_status} to {new_status}",
        "order_id": order_id,
        "product_id": product_id,
        "old_status": old_status,
        "new_status": new_status,
        "current_inventory": products[product_id]["inventory_count"] if product_id in products else "unknown"
    }

def cancel_order(order_id):
    """
    Cancels an order and adjusts inventory if needed.
    """
    # Simply call update_order_status with "Canceled" status
    return update_order_status(order_id, "Canceled")

# Modified product information tool to support fuzzy search
def get_product_info(product_id_or_name):
    """
    Get information about a product by its ID or name.
    Supports fuzzy matching for product names.

    Args:
        product_id_or_name (str): Product ID (e.g., "P1") or name (e.g., "Widget")

    Returns:
        dict: Product information if found, error message otherwise
    """
    # Check if it's a product ID
    if product_id_or_name in products:
        return {
            "status": "success",
            "product_id": product_id_or_name,
            "product": products[product_id_or_name]
        }

    # Try fuzzy matching by name
    product_id, product = find_product_by_name(product_id_or_name)

    if product_id:
        return {
            "status": "success",
            "message": f"Found product matching '{product_id_or_name}'",
            "product_id": product_id,
            "product": product
        }

    return {
        "status": "error",
        "message": f"No product found matching '{product_id_or_name}'"
    }

def list_all_products():
    """
    List all available products in the inventory.

    Returns:
        dict: All products with their details
    """
    return {
        "status": "success",
        "count": len(products),
        "products": products
    }

In [71]:
# Class to manage conversation context
class ConversationContext:
    def __init__(self):
        # Initialize conversation history and context store
        self.messages: List[Dict[str, Any]] = []
        self.context: Dict[str, Any] = {
            "customers": {},     # Recently viewed/mentioned customers
            "products": {},      # Recently viewed/mentioned products
            "orders": {},        # Recently viewed/mentioned orders
            "last_action": None  # Last action performed by the user
        }
        self.session_start_time = datetime.now()

    def add_user_message(self, message: str) -> None:
        """Add a user message to the conversation history"""
        self.messages.append({"role": "user", "content": message})

    def add_assistant_message(self, message: str) -> None:
        """Add an assistant message to the conversation history"""
        self.messages.append({"role": "assistant", "content": message})

    def update_context(self, entity_type: str, entity_id: str, data: Any) -> None:
        """Update the conversation context with information about an entity"""
        if entity_type in self.context:
            self.context[entity_type][entity_id] = data

    def set_last_action(self, action_type: str, action_data: Any) -> None:
        """Set the last action performed by the user"""
        self.context["last_action"] = {
            "type": action_type,
            "data": action_data,
            "timestamp": datetime.now().isoformat()
        }

    def get_full_conversation_for_claude(self) -> List[Dict[str, Any]]:
        """Get the full conversation history in a format suitable for Claude API"""
        # Return only the messages, not the context
        # Context will be provided separately in the system prompt
        return self.messages.copy()

    def get_context_summary(self) -> str:
        """Generate a summary of the current context for inclusion in the system prompt"""
        summary_parts = []

        # Add information about recent customers
        if self.context["customers"]:
            customer_info = []
            for cid, customer in self.context["customers"].items():
                if isinstance(customer, dict) and "name" in customer:
                    customer_info.append(f"Customer {cid}: {customer['name']}")
                else:
                    customer_info.append(f"Customer {cid}")
            summary_parts.append("Recent customers: " + ", ".join(customer_info))

        # Add information about recent products
        if self.context["products"]:
            product_info = []
            for pid, product in self.context["products"].items():
                if isinstance(product, dict) and "name" in product:
                    product_info.append(f"Product {pid}: {product['name']}")
                else:
                    product_info.append(f"Product {pid}")
            summary_parts.append("Recent products: " + ", ".join(product_info))

        # Add information about recent orders
        if self.context["orders"]:
            order_info = []
            for oid, order in self.context["orders"].items():
                if isinstance(order, dict):
                    if "product_name" in order and "quantity" in order:
                        order_info.append(f"Order {oid}: {order['quantity']} x {order['product_name']} (Status: {order.get('status', 'Unknown')})")
                    else:
                        order_info.append(f"Order {oid}")
                else:
                    order_info.append(f"Order {oid}")
            summary_parts.append("Recent orders: " + ", ".join(order_info))

        # Add information about the last action
        if self.context["last_action"]:
            action = self.context["last_action"]
            if action["type"] == "create_order":
                order_data = action["data"]
                if isinstance(order_data, dict) and "order_id" in order_data:
                    order_id = order_data["order_id"]
                    summary_parts.append(f"Last action: Created order {order_id}")
            elif action["type"] == "update_product":
                product_data = action["data"]
                if isinstance(product_data, dict) and "product_id" in product_data:
                    product_id = product_data["product_id"]
                    summary_parts.append(f"Last action: Updated product {product_id}")
            else:
                summary_parts.append(f"Last action: {action['type']}")

        return "\n".join(summary_parts)

    def clear(self) -> None:
        """Clear the conversation history and context"""
        self.messages = []
        self.context = {
            "customers": {},
            "products": {},
            "orders": {},
            "last_action": None
        }
        self.session_start_time = datetime.now()

# Initialize the conversation context
conversation_context = ConversationContext()

In [72]:
def contextual_chatbot_interaction(user_message: str) -> str:
    """
    Interact with the chatbot while maintaining conversation context

    Args:
        user_message: The user's message

    Returns:
        The chatbot's response
    """
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    # Add the user's message to the conversation context
    conversation_context.add_user_message(user_message)

    # Get the current context summary
    context_summary = conversation_context.get_context_summary()

    # Create the messages to send to Claude, including conversation history
    full_system_prompt = system_prompt
    if context_summary:
        full_system_prompt += f"\n\nCurrent context:\n{context_summary}"

    # Get the conversation history
    messages = conversation_context.get_full_conversation_for_claude()

    # Make the initial API call
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        system=full_system_prompt,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    # Track all the tools used in this interaction for context updating
    tools_used = []

    # Handle tool calls if needed
    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        # Process the tool call
        tool_result = process_tool_call(tool_name, tool_input)

        # Track the tool usage for context updating
        tools_used.append({
            "name": tool_name,
            "input": tool_input,
            "result": tool_result
        })

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        # Update the messages with the tool results
        messages = conversation_context.get_full_conversation_for_claude()
        messages.append({"role": "assistant", "content": response.content})
        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": str(tool_result),
                }
            ],
        })

        # Make another API call with the updated messages
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            system=full_system_prompt,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    # Extract the final text response
    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    # Add the assistant's response to the conversation context
    conversation_context.add_assistant_message(final_response)

    # Update context based on tools used
    for tool_usage in tools_used:
        tool_name = tool_usage["name"]
        tool_input = tool_usage["input"]
        tool_result = tool_usage["result"]

        # Handle different types of tools and update context accordingly
        if tool_name == "get_customer_info" and isinstance(tool_result, dict) and "name" in tool_result:
            customer_id = tool_input.get("customer_id")
            if customer_id:
                conversation_context.update_context("customers", customer_id, tool_result)

        elif tool_name == "get_product_info" and isinstance(tool_result, dict) and "status" in tool_result and tool_result["status"] == "success":
            product_id = tool_result.get("product_id")
            product = tool_result.get("product")
            if product_id and product:
                conversation_context.update_context("products", product_id, product)

        elif tool_name == "get_order_details" and isinstance(tool_result, dict) and "id" in tool_result:
            order_id = tool_input.get("order_id")
            if order_id:
                conversation_context.update_context("orders", order_id, tool_result)

        elif tool_name == "create_order" and isinstance(tool_result, dict) and "status" in tool_result and tool_result["status"] == "success":
            order_id = tool_result.get("order_id")
            order_details = tool_result.get("order_details")
            if order_id and order_details:
                conversation_context.update_context("orders", order_id, order_details)
                conversation_context.set_last_action("create_order", tool_result)

        elif tool_name == "update_product" and isinstance(tool_result, dict) and "status" in tool_result and tool_result["status"] == "success":
            product_id = tool_result.get("product_id")
            product = tool_result.get("product")
            if product_id and product:
                conversation_context.update_context("products", product_id, product)
                conversation_context.set_last_action("update_product", tool_result)

        elif tool_name == "update_order_status" and isinstance(tool_result, dict) and "status" in tool_result and tool_result["status"] == "success":
            order_id = tool_result.get("order_id")
            if order_id:
                # Need to fetch the updated order details
                updated_order = get_order_details(order_id)
                if isinstance(updated_order, dict) and "id" in updated_order:
                    conversation_context.update_context("orders", order_id, updated_order)
                conversation_context.set_last_action("update_order_status", tool_result)

    return final_response

# Legacy function maintained for backward compatibility
def chatbot_interaction(user_message):
    return contextual_chatbot_interaction(user_message)


'\n# Conversation with context awareness\ncontextual_chatbot_interaction("Show me all available products")\ncontextual_chatbot_interaction("I\'d like to order 3 of the widget please")\ncontextual_chatbot_interaction("What\'s the status of my order?")\ncontextual_chatbot_interaction("Can you change that to 5 instead of 3?")\ncontextual_chatbot_interaction("When will my order ship?")\n\n# Clear context when starting a new session\nconversation_context.clear()\n'

In [86]:
# Clear context when starting a new session
conversation_context.clear()


In [76]:
contextual_chatbot_interaction("Show me all available products")


User Message: Show me all available products

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll show you all the products we currently have in our inventory. Let me retrieve that information for you.", type='text'), ToolUseBlock(id='toolu_01PaTE2C1NV2a9u6vHD3ErxV', input={}, name='list_all_products', type='tool_use')]

Tool Used: list_all_products
Tool Input:
{}

Tool Result:
{
  "status": "success",
  "count": 3,
  "products": {
    "P1": {
      "name": "Widget A",
      "description": "A simple widget. Very compact.",
      "price": 19.99,
      "inventory_count": 999
    },
    "P2": {
      "name": "Gadget B",
      "description": "A powerful gadget. It spins.",
      "price": 49.99,
      "inventory_count": 200
    },
    "P3": {
      "name": "Perplexinator",
      "description": "A perplexing perfunctator",
      "price": 79.99,
      "inventory_count": 1483
    }
  }
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, te

"Here are all the products currently available in our inventory:\n\n1. **Widget A** - $19.99\n   - Description: A simple widget. Very compact.\n   - In stock: 999 units\n\n2. **Gadget B** - $49.99\n   - Description: A powerful gadget. It spins.\n   - In stock: 200 units\n\n3. **Perplexinator** - $79.99\n   - Description: A perplexing perfunctator\n   - In stock: 1483 units\n\nIs there any specific product you'd like more information about or would you like to place an order for any of these items?"

In [77]:
contextual_chatbot_interaction("I'd like to order 3 of the widget please")


User Message: I'd like to order 3 of the widget please

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'd be happy to place that order for you. Let me process an order for 3 units of Widget A.", type='text'), ToolUseBlock(id='toolu_01Q2ghurb4QchyZwmcDNcoep', input={'product_id_or_name': 'Widget A', 'quantity': 3, 'status': 'Processing'}, name='create_order', type='tool_use')]

Tool Used: create_order
Tool Input:
{
  "product_id_or_name": "Widget A",
  "quantity": 3,
  "status": "Processing"
}

Tool Result:
{
  "status": "success",
  "order_id": "O4",
  "order_details": {
    "id": "O4",
    "product": "P1",
    "product_name": "Widget A",
    "quantity": 3,
    "price": 19.99,
    "status": "Processing"
  },
  "remaining_inventory": 999
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Great! Your order for 3 units of Widget A has been successfully placed. \n\nHere's your order summary:\n- Order ID: O4\n- Product: Widget A

"Great! Your order for 3 units of Widget A has been successfully placed. \n\nHere's your order summary:\n- Order ID: O4\n- Product: Widget A\n- Quantity: 3\n- Price per unit: $19.99\n- Total amount: $59.97\n- Current status: Processing\n\nYour order is now being processed. Is there anything else you would like to know about your order or can I help you with something else?"

In [78]:
contextual_chatbot_interaction("What's the status of my order?")


User Message: What's the status of my order?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text='Let me check the status of your order for you.', type='text'), ToolUseBlock(id='toolu_0177V3CcgAFzvmQZz63SpKeP', input={'order_id': 'O4'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O4"
}

Tool Result:
{
  "id": "O4",
  "product": "P1",
  "product_name": "Widget A",
  "quantity": 3,
  "price": 19.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='Your order (Order ID: O4) for 3 Widget A units is currently in "Processing" status. This means our team is preparing your order for shipment. Once your order has been shipped, the status will update to "Shipped."\n\nIs there anything else you would like to know about your order, or can I help you with something else today?', type='text')]

Final Response: Your order (Order ID: O4) for 3 Widget A units is

'Your order (Order ID: O4) for 3 Widget A units is currently in "Processing" status. This means our team is preparing your order for shipment. Once your order has been shipped, the status will update to "Shipped."\n\nIs there anything else you would like to know about your order, or can I help you with something else today?'

In [79]:
contextual_chatbot_interaction("Can you change that to 5 instead of 3?")


User Message: Can you change that to 5 instead of 3?

Initial Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="I understand you'd like to change your order quantity from 3 to 5 Widget A units. Unfortunately, our system doesn't have a direct function to modify the quantity of an existing order.\n\nI can help you with two options:\n1. Cancel your current order and create a new one with 5 units\n2. Keep your current order for 3 units and create an additional order for 2 more units\n\nWhich option would you prefer?", type='text')]

Final Response: I understand you'd like to change your order quantity from 3 to 5 Widget A units. Unfortunately, our system doesn't have a direct function to modify the quantity of an existing order.

I can help you with two options:
1. Cancel your current order and create a new one with 5 units
2. Keep your current order for 3 units and create an additional order for 2 more units

Which option would you prefer?


"I understand you'd like to change your order quantity from 3 to 5 Widget A units. Unfortunately, our system doesn't have a direct function to modify the quantity of an existing order.\n\nI can help you with two options:\n1. Cancel your current order and create a new one with 5 units\n2. Keep your current order for 3 units and create an additional order for 2 more units\n\nWhich option would you prefer?"

In [80]:
contextual_chatbot_interaction("Let's go with option 2")


User Message: Let's go with option 2

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you place an additional order for 2 more Widget A units. Let me take care of that for you:", type='text'), ToolUseBlock(id='toolu_01RaxqfcufHPtfy3RdhMFHM3', input={'product_id_or_name': 'Widget A', 'quantity': 2, 'status': 'Processing'}, name='create_order', type='tool_use')]

Tool Used: create_order
Tool Input:
{
  "product_id_or_name": "Widget A",
  "quantity": 2,
  "status": "Processing"
}

Tool Result:
{
  "status": "success",
  "order_id": "O5",
  "order_details": {
    "id": "O5",
    "product": "P1",
    "product_name": "Widget A",
    "quantity": 2,
    "price": 19.99,
    "status": "Processing"
  },
  "remaining_inventory": 999
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Great! I've placed an additional order for 2 more Widget A units. \n\nHere's your new order summary:\n- Order ID: O5\n- Product: Widget A  \n- Quan

"Great! I've placed an additional order for 2 more Widget A units. \n\nHere's your new order summary:\n- Order ID: O5\n- Product: Widget A  \n- Quantity: 2\n- Price per unit: $19.99\n- Total amount: $39.98\n- Current status: Processing\n\nThis is in addition to your existing order (O4) for 3 Widget A units that's already processing. So in total, you'll be receiving 5 Widget A units across these two orders.\n\nIs there anything else you'd like assistance with today?"

In [81]:
contextual_chatbot_interaction("How many widgets are left in stock?")


User Message: How many widgets are left in stock?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text='Let me check the current inventory for Widget A for you.', type='text'), ToolUseBlock(id='toolu_01QiMUpyfBzsSvg2xUaSH9YV', input={'product_id_or_name': 'Widget A'}, name='get_product_info', type='tool_use')]

Tool Used: get_product_info
Tool Input:
{
  "product_id_or_name": "Widget A"
}

Tool Result:
{
  "status": "success",
  "message": "Found product matching 'Widget A'",
  "product_id": "P1",
  "product": {
    "name": "Widget A",
    "description": "A simple widget. Very compact.",
    "price": 19.99,
    "inventory_count": 999
  }
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="After your orders (3 units in your first order and 2 units in your second order), there are currently 999 Widget A units remaining in stock. \n\nWidget A is well-stocked, so there's no need to worry about availability. Is there anything else you'd

"After your orders (3 units in your first order and 2 units in your second order), there are currently 999 Widget A units remaining in stock. \n\nWidget A is well-stocked, so there's no need to worry about availability. Is there anything else you'd like to know about the product or your orders?"

In [82]:
contextual_chatbot_interaction("How many widgets are left in stock after you ship my order?")


User Message: How many widgets are left in stock after you ship my order?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text='Let me check the current inventory status for Widget A based on your orders.', type='text'), ToolUseBlock(id='toolu_01GUFTq9bnywpwyUzghSPeGV', input={'product_id_or_name': 'Widget A'}, name='get_product_info', type='tool_use')]

Tool Used: get_product_info
Tool Input:
{
  "product_id_or_name": "Widget A"
}

Tool Result:
{
  "status": "success",
  "message": "Found product matching 'Widget A'",
  "product_id": "P1",
  "product": {
    "name": "Widget A",
    "description": "A simple widget. Very compact.",
    "price": 19.99,
    "inventory_count": 999
  }
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='After your orders are shipped, there will be 994 Widget A units remaining in stock. This is calculated as:\n\nCurrent inventory (999 units) - Your first order (3 units) - Your second order (2 units) = 99

'After your orders are shipped, there will be 994 Widget A units remaining in stock. This is calculated as:\n\nCurrent inventory (999 units) - Your first order (3 units) - Your second order (2 units) = 994 units\n\nThe 999 units shown is the current inventory count, and since both of your orders are still in "Processing" status, the inventory hasn\'t been reduced yet. When the orders change to "Shipped" status, the inventory will be updated accordingly.\n\nIs there anything else you\'d like to know?'

In [83]:
contextual_chatbot_interaction("Please ship my order now")


User Message: Please ship my order now

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text='I\'ll update the status of your orders to "Shipped" right away. Let me do that for you.', type='text'), ToolUseBlock(id='toolu_01URcpMGyDx5KpKUxULMSDeN', input={'order_id': 'O4', 'new_status': 'Shipped'}, name='update_order_status', type='tool_use')]

Tool Used: update_order_status
Tool Input:
{
  "order_id": "O4",
  "new_status": "Shipped"
}

Tool Result:
{
  "status": "success",
  "message": "Order O4 status updated from Processing to Shipped",
  "order_id": "O4",
  "product_id": "P1",
  "old_status": "Processing",
  "new_status": "Shipped",
  "current_inventory": 996
}

Response:
Stop Reason: tool_use
Content: [ToolUseBlock(id='toolu_017c4EzcZWw72ZoF51A9muVJ', input={'order_id': 'O5', 'new_status': 'Shipped'}, name='update_order_status', type='tool_use')]

Tool Used: update_order_status
Tool Input:
{
  "order_id": "O5",
  "new_status": "Shipped"
}

Tool Result:


'I\'ve updated your most recent order (Order ID: O5) for 2 Widget A units to "Shipped" status. It\'s now on its way to you!\n\nWould you like me to ship your other order (Order ID: O4) for 3 Widget A units as well?'